# Piecewise-Linear Regression

### `SMC` approach

In [ ]:
# import libraries
from scipy.special import softmax

## basic imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
import seaborn as sns
import cvxpy as cp
import time
clear_output()
myseed = '150824' 
np.random.seed(int(myseed))

In [ ]:
## problem dimensions

B1 = int(6) 
B2 = int(5) 
L = 100.0 # X search space dim

AUGMENTED = True
RESCALE = True
TITLE = 'insurance' 
TARGET_NAME = 'charges' 

### utils

In [ ]:
def proj_simplex_vec(V, z=1):
    n_features = V.shape[1]
    U = np.sort(V, axis=1)[:, ::-1]
    z = np.ones(len(V)) * z
    cssv = np.cumsum(U, axis=1) - z[:, np.newaxis]
    ind = np.arange(n_features) + 1
    cond = U - cssv / ind > 0
    rho = np.count_nonzero(cond, axis=1)
    theta = cssv[np.arange(len(V)), rho - 1] / rho
    return np.maximum(V - theta[:, np.newaxis], 0)

In [ ]:
'''
utils (global)
'''    

def extended_omegas(omegas,augmented=False):
    if augmented:
        N,d = omegas.shape
        embedding = np.zeros((N,int(d+d*(d+1)/2)))
        for s,samp in enumerate(omegas):
            new_omega = list(samp)
            buf = np.outer(samp,samp)
            for ofd in range(d):
                new_omega += list(np.diagonal(buf,ofd))
            embedding[s] = np.array(new_omega)
    else:
        embedding = omegas.copy()
    return np.hstack((embedding,np.ones((len(omegas),1))))

def rescale(data):
    if len(data.shape)==2:
        num_samples,num_features = data.shape
        new_data = data.copy()
        means,stds = [],[]
        for colid,feat in enumerate(data.T):
            mfeat,stdfeat = np.mean(feat),np.std(feat)
            means.append(mfeat)
            stds.append(stdfeat)
            if stdfeat>0:
                new_data[:,colid] = (feat-mfeat)/stdfeat
            else:
                means[colid] = 0
                stds[colid] = 1
        return new_data,means,stds
    else:
        mean = np.mean(data)
        std = np.std(data)
        if std>0:
            return (data-mean)/std,mean,std
        else:
            return data,0,1

def coordinates(sigma_,B1_=B1,B2_=B2):
    assert sigma_>=0 and sigma_<=B1_*B2_-1, 'value range error (0 -> H-1)'
    e2 = sigma_%B2_
    e1 = (sigma_-(e2-1))/B2_
    return (int(e1),int(e2))

def selector(coordinates,B1_=B1,B2_=B2):
    return coordinates[1]+coordinates[0]*B2_

def recast(x,w_ext,B1_=B1,B2_=B2):
    assert len(x)==(B1_+B2_)*w_ext, 'dimension error'
    mat = x.reshape((B1_+B2_,w_ext))
    return mat[:B1_],mat[B1_:]

def mix(mat1,mat2):
    assert mat1.shape[1]==mat2.shape[1],'dimension error'
    buf = []
    for lm1 in mat1:
        for lm2 in mat2:
            buf.append(list(lm1+lm2))
    return np.array(buf)

def unfold(mat1,mat2):
    return (np.vstack((mat1,mat2))).flatten()

### true data generation

In [ ]:
def preprocess(fname,tf,augmented=True):
    ## loading part 
    try:
        raw_df = pd.read_csv(fname)
        raw_df.dropna(inplace=True)
        if isinstance(tf,str):
            target_df = raw_df[tf]
        elif isinstance(tf,numbers.Number):
            target_df = raw_df.iloc[:,int(tf)]
        else:
            print('tf should either be a column name (str) or location (int)')
            return None,None
        target = target_df.values
        data_df = raw_df.drop(columns=[tf])
        obj_df = data_df.select_dtypes(include='object')
        to_categorize = list(obj_df.columns)
        for feat_str in to_categorize:
            if isinstance(obj_df[feat_str].values[0],str):
                cats = obj_df[feat_str].unique()
                ncats = len(cats)
                if ncats%2==0:
                    vals = np.linspace(-int(ncats/2),int(ncats/2),ncats)
                else:
                    vals = np.linspace(-int((ncats-1)/2),int((ncats)/2),ncats)
                vals /= max(abs(vals))
                dic_rep = {}
                for cat,num in zip(cats,vals):
                    dic_rep[cat] = num
                data_df[feat_str].replace(dic_rep,inplace=True)
        data = data_df.select_dtypes('number').values
    except:
        print('-> error while loading your file')
        return None,None
        
    data = extended_omegas(data,augmented) # adds "bias terms"
    
    return target,data

In [ ]:
pre_taus,pre_omegas = preprocess(TITLE+'.csv',TARGET_NAME,AUGMENTED)
if RESCALE:
    taus,meantaus,stdtaus = rescale(pre_taus)
    omegas,meanomegas,stdomegas = rescale(pre_omegas)
else:
    taus = pre_taus.copy()
    omegas = omegas.copy()

In [ ]:
N_train = min(len(omegas),int(750))
id_train = np.random.choice(np.arange(len(omegas)),replace=False,size=N_train)

omegas_train = omegas[id_train]
taus_train = taus[id_train]
w_ext = omegas_train.shape[1]

In [ ]:
w_ext

In [ ]:
N_train

### TRAINING

`parametric LP` 

In [ ]:
# symmetry breakers tolerances
margin = .0

# variables
mat1_cvx = cp.Variable((B1,w_ext))
mat2_cvx = cp.Variable((B2,w_ext))

# parameters
param1 = cp.Parameter((B1,w_ext))
param2 = cp.Parameter((B2,w_ext))

# basic feasible set 
X = [cp.norm(mat1_cvx[e1][:-1],'inf')<=L for e1 in range(B1)] + [cp.norm(mat2_cvx[e2][:-1],'inf')<=L for e2 in range(B2)] 

# prior-knowledge encoding
symmetry_breaks = []
if B1>1:
    for e1 in range(B1-1):
        symmetry_breaks += [cp.sum(mat1_cvx[e1])+margin<=cp.sum(mat1_cvx[e1+1])]
if B2>1:
    for e2 in range(B2-1):
        symmetry_breaks += [cp.sum(mat2_cvx[e2])+margin<=cp.sum(mat2_cvx[e2+1])]

# objective function implementation
val1_cvx,val2_cvx = mat1_cvx@omegas_train.T,mat2_cvx@omegas_train.T
mval1_cvx,mval2_cvx = cp.max(val1_cvx,0),cp.max(val2_cvx,0)
vec_h_bar_cvx = cp.maximum(taus_train+mval2_cvx,mval1_cvx)+cp.maximum(-taus_train+mval1_cvx,mval2_cvx)
h_bar_smc = 1/N_train * cp.sum(vec_h_bar_cvx)
F_param_smc = h_bar_smc-(1/N_train)*(cp.sum(cp.multiply(param1,mat1_cvx))\
                                               +cp.sum(cp.multiply(param2,mat2_cvx)))

prob_param_smc = cp.Problem(cp.Minimize(F_param_smc),X+symmetry_breaks)

In [ ]:
'''-> ERM evaluation<-'''
def Feval(mat1_val,mat2_val,data=omegas_train,target=taus_train):
    val1,val2=mat1_val@data.T,mat2_val@data.T
    return np.mean(np.abs(target-(np.max(val1,0)-np.max(val2,0))))

'''-> h components evaluation <-'''
def heval(mat1_val,mat2_val,data=omegas_train):
    loc_B1,loc_B2 = len(mat1_val),len(mat2_val)
    val1,val2=mat1_val@data.T,mat2_val@data.T
    buf = np.zeros((len(data),loc_B1*loc_B2))
    for e1,v1 in enumerate(val1):
        buf[:,e1*loc_B2:(e1+1)*loc_B2] = -(np.outer(v1,np.ones(loc_B2))+val2.T)
    return buf

In [ ]:
'''-> computation of parameters <-
weights ndarray of shape (N_train,B1*B2)
'''
def w2p(weights):
    param1_val = np.zeros((B1,w_ext))
    param2_val = np.zeros((B2,w_ext))
    for l in range(B1*B2):
        e1_sel,e2_sel = coordinates(l)
        vec_shift = weights[:,l]@omegas_train
        param1_val[e1_sel]+=vec_shift
        param2_val[e2_sel]+=vec_shift
    param1.value = param1_val
    param2.value = param2_val
    
    
''' '''
def Q_opt(vals):
    buf = np.zeros(vals.shape)
    for s,sigma_s in enumerate(np.argmin(vals,1)):
        buf[s,sigma_s] += 1
    return buf

def Q_safe(vals,Q_current,Q_hat,Q_star,C):
    Q_next = np.zeros(Q_star.shape)
    combinations = np.zeros(len(Q_current))
    for _,Q_hat_elem in enumerate(Q_hat):
        num = np.sum((Q_current[_]-Q_star[_])*vals[_])
        denum = np.sum((Q_hat_elem-Q_star[_])*vals[_])
        if C*num>=denum:
            combinations[_] = 1
        else:
            combinations[_] = min(1,max(0,C*(num/max(1e-9,denum))))
        Q_next[_] = combinations[_]*Q_hat_elem+(1-combinations[_])*Q_star[_]
    return Q_next,combinations

In [ ]:
NREPS = int(50)
Q_init_mats = []
for rep in range(NREPS):
    Q_init = np.random.uniform(0,1,(N_train,B1*B2))
    Q_init = np.array([q/np.sum(q) for q in Q_init])
    Q_init_mats.append(Q_init)

N_max = int(400)
tol = 1e-8

In [ ]:
'''MAXMIN'''

def Q_hat_mm(vals,param=1):
    minvals,maxvals = np.min(vals,1),np.max(vals,1)
    return proj_simplex_vec(param*(np.outer(maxvals,np.ones(vals.shape[1]))-np.array(vals))/(np.outer(maxvals,np.ones(vals.shape[1]))-np.outer(minvals,np.ones(vals.shape[1]))))

## init
f_maxmin_overallbest = np.inf
fval_maxmin = []
t_maxmin = []
mat1_maxmin_out,mat2_maxmin_out = None,None
Q_maxmin_out = None


print('=> maxmin (START)')

param_schedule = lambda k: k**(2/3) 
C_schedule = lambda k: 2/(k**(1/2)+3) # 0 for am

for rep in range(NREPS):
    
    print('REPID #'+str(rep+1))
    print(' ')
    
    Q = Q_init_mats[rep].copy()
    w2p(Q)
    ref_val = np.inf
    
    t_init_maxmin = time.time()
    
    try:

        ## main loop
        for k in range(N_max):

            C,kappa = C_schedule(k),param_schedule(k)

            #### set x 
            prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

            #### set weights
            components_ = heval(mat1_cvx.value,mat2_cvx.value)
            prev = np.sum(components_/N_train*Q)
            Q_hat = Q_hat_mm(components_,param=kappa)
            Q_star = Q_opt(components_)
            Q,combis = Q_safe(components_,Q,Q_hat,Q_star,C)
            w2p(Q)
            post = np.sum(components_/N_train*Q)
            F_Q_plus_x_plus = F_param_smc.value
            G_crit = max(0,prev-post)
            F_ = Feval(mat1_cvx.value,mat2_cvx.value)
            print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e | avg. epsilon. %4.4e" % (k + 1, F_,F_Q_plus_x_plus,G_crit,np.mean(combis)))
            if (ref_val-F_)<tol and k>1:
                print('=> convergence to criticality ... polishing phase')
                Q_final_maxmin = Q_opt(heval(mat1_cvx.value,mat2_cvx.value))
                w2p(Q_final_maxmin)
                prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                F_ = Feval(mat1_cvx.value,mat2_cvx.value)
                t_maxmin.append(time.time()-t_init_maxmin)
                fval_maxmin.append(F_)
                print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                print('=> maxmin (END)')
                break;
            else:
                ref_val = F_Q_plus_x_plus
        if k==N_max-1:
            fval_maxmin.append(Feval(mat1_cvx.value,mat2_cvx.value))
            t_maxmin.append(time.time()-t_init_maxmin)
    except: 
        print('... solver failed | early interruption ')
        fval_maxmin.append(Feval(mat1_cvx.value,mat2_cvx.value))
        t_maxmin.append(time.time()-t_init_maxmin)
    print(' ')
    if fval_maxmin[-1]<=f_maxmin_overallbest:
        print(' /!\ NEW BEST | F = '+str(fval_maxmin[-1])+'  /!\ ')
        f_maxmin_overallbest = fval_maxmin[-1]
        mat1_maxmin_out = mat1_cvx.value
        mat2_maxmin_out = mat2_cvx.value
        Q_maxmin_out = Q_final_maxmin
        print(' ')

In [ ]:
'''AM'''

## init
f_am_overallbest = np.inf
fval_am = []
t_am = []
mat1_am_out,mat2_am_out = None,None
Q_am_out = None


print('=> am (START)')

for rep in range(NREPS):
    
    print('REPID #'+str(rep+1))
    print(' ')
    
    Q = Q_init_mats[rep].copy()
    w2p(Q)
    ref_val = np.inf
    
    t_init_am = time.time()
    
    try:

        ## main loop
        for k in range(N_max):

            #### set x 
            prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

            #### set weights
            components_ = heval(mat1_cvx.value,mat2_cvx.value)
            prev = np.sum(components_/N_train*Q)
            Q = Q_opt(components_)
            w2p(Q)
            post = np.sum(components_/N_train*Q)
            F_Q_plus_x_plus = F_param_smc.value
            G_crit = max(0,prev-post)
            F_ = Feval(mat1_cvx.value,mat2_cvx.value)
            print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e " % (k + 1, F_,F_Q_plus_x_plus,G_crit))
            if (ref_val-F_)<tol and k>1:
                print('=> convergence to criticality ... polishing phase')
                Q_final_am = Q_opt(heval(mat1_cvx.value,mat2_cvx.value))
                w2p(Q_final_am)
                prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                F_ = Feval(mat1_cvx.value,mat2_cvx.value)
                t_am.append(time.time()-t_init_am)
                fval_am.append(F_)
                print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                print('=> am (END)')
                break;
            else:
                ref_val = F_Q_plus_x_plus
        if k==N_max-1:
            fval_am.append(Feval(mat1_cvx.value,mat2_cvx.value))
            t_am.append(time.time()-t_init_am)
    except: 
        print('... solver failed | early interruption ')
        fval_am.append(Feval(mat1_cvx.value,mat2_cvx.value))
        t_am.append(time.time()-t_init_am)
    print(' ')
    if fval_am[-1]<=f_am_overallbest:
        print(' /!\ NEW BEST | F = '+str(fval_am[-1])+'  /!\ ')
        f_am_overallbest = fval_am[-1]
        mat1_am_out = mat1_cvx.value
        mat2_am_out = mat2_cvx.value
        Q_am_out = Q_final_am
        print(' ')

In [ ]:
'''Baratt-Boyd'''

def Q_hat_bb(vals,Q,param=1):
    minids = np.argmin(vals,1)
    Q_hat = Q.copy()
    for s,sigma_s in enumerate(minids):
        Q_hat[s] -= param*np.ones(vals.shape[1])
        Q_hat[s,sigma_s] += 2*param
    return proj_simplex_vec(Q_hat)

## init
f_bb_overallbest = np.inf
fval_bb = []
t_bb = []
mat1_bb_out,mat2_bb_out = None,None
Q_bb_out = None


print('=> baratt-boyd (START)')

param_schedule = lambda k: 1e-1

for rep in range(NREPS):
    
    print('REPID #'+str(rep+1))
    print(' ')
    
    Q = Q_init_mats[rep].copy()
    w2p(Q)
    ref_val = np.inf
    
    t_init_bb = time.time()
    
    try:

        ## main loop
        for k in range(N_max):

            kappa = param_schedule(k)

            #### set x 
            prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

            #### set weights
            components_ = heval(mat1_cvx.value,mat2_cvx.value)
            prev = np.sum(components_/N_train*Q)
            Q = Q_hat_bb(components_,Q,param=kappa)
            w2p(Q)
            post = np.sum(components_/N_train*Q)
            F_Q_plus_x_plus = F_param_smc.value
            G_crit = max(0,prev-post)
            F_ = Feval(mat1_cvx.value,mat2_cvx.value)
            print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e " % (k + 1, F_,F_Q_plus_x_plus,G_crit))
            if (ref_val-F_)<tol and k>np.floor(1/param_schedule(0)): # /!\ otherwise early unwanted convergence /!\
                print('=> convergence to criticality ... polishing phase')
                Q_final_bb = Q_opt(heval(mat1_cvx.value,mat2_cvx.value))
                w2p(Q_final_bb)
                prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                F_ = Feval(mat1_cvx.value,mat2_cvx.value)
                t_bb.append(time.time()-t_init_bb)
                fval_bb.append(F_)
                print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                print('=> maxmin (END)')
                break;
            else:
                ref_val = F_Q_plus_x_plus
        if k==N_max-1:
            fval_bb.append(Feval(mat1_cvx.value,mat2_cvx.value))
            t_bb.append(time.time()-t_init_bb)
    except: 
        print('... solver failed | early interruption ')
        fval_bb.append(Feval(mat1_cvx.value,mat2_cvx.value))
        t_bb.append(time.time()-t_init_bb)
    print(' ')
    if fval_bb[-1]<=f_bb_overallbest:
        print(' /!\ NEW BEST | F = '+str(fval_bb[-1])+'  /!\ ')
        f_bb_overallbest = fval_bb[-1]
        mat1_bb_out = mat1_cvx.value
        mat2_bb_out = mat2_cvx.value
        Q_bb_out = Q_final_bb
        print(' ')

In [ ]:
'''Softmin'''

def Q_hat_softmin(vals,param=1,num_pert_inf=5e-7):
    pert_vals = vals+np.random.uniform(-num_pert_inf,num_pert_inf,vals.shape)
    return softmax(-param*pert_vals/np.maximum(1e-4,np.abs(np.outer(np.mean(vals,1),np.ones(B1*B2)))),axis=1)

## init
f_sm_overallbest = np.inf
fval_sm = []
t_sm = []
mat1_sm_out,mat2_sm_out = None,None
Q_sm_out = None


print('=> softmin (START)')

param_schedule = lambda k: (3/2)**(k**(3/4)) if (3/2)**(k**(3/4))<1e16 else np.inf
C_schedule = lambda k: 2/(k**(1/2)+3)

for rep in range(NREPS):
    
    print('REPID #'+str(rep+1))
    print(' ')
    
    Q = Q_init_mats[rep].copy()
    w2p(Q)
    ref_val = np.inf
    
    t_init_sm = time.time()
    
    try:

        ## main loop
        for k in range(N_max):

            C,kappa = C_schedule(k),param_schedule(k)

            #### set x 
            prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

            #### set weights
            components_ = heval(mat1_cvx.value,mat2_cvx.value)
            prev = np.sum(components_/N_train*Q)
            Q_hat = Q_hat_softmin(components_,param=kappa)
            Q_hat = Q_hat/np.outer(np.sum(Q,1),np.ones(B1*B2))
            Q_star = Q_opt(components_)
            Q,combis = Q_safe(components_,Q,Q_hat,Q_star,C)
            w2p(Q)
            post = np.sum(components_/N_train*Q)
            F_Q_plus_x_plus = F_param_smc.value
            G_crit = max(0,prev-post)
            F_ = Feval(mat1_cvx.value,mat2_cvx.value)
            print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e | avg. epsilon. %4.4e" % (k + 1, F_,F_Q_plus_x_plus,G_crit,np.mean(combis)))
            if (ref_val-F_)<tol and k>1:
                print('=> convergence to criticality ... polishing phase')
                Q_final_sm = Q_opt(heval(mat1_cvx.value,mat2_cvx.value))
                w2p(Q_final_sm)
                prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                F_ = Feval(mat1_cvx.value,mat2_cvx.value)
                t_sm.append(time.time()-t_init_sm)
                fval_sm.append(F_)
                print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                print('=> softmin (END)')
                break;
            else:
                ref_val = F_Q_plus_x_plus
        if k==N_max-1:
            fval_sm.append(Feval(mat1_cvx.value,mat2_cvx.value))
            t_sm.append(time.time()-t_init_sm)
    except: 
        print('... solver failed | early interruption ')
        fval_sm.append(Feval(mat1_cvx.value,mat2_cvx.value))
        t_sm.append(time.time()-t_init_sm)
    print(' ')
    if fval_sm[-1]<=f_sm_overallbest:
        print(' /!\ NEW BEST | F = '+str(fval_sm[-1])+'  /!\ ')
        f_sm_overallbest = fval_sm[-1]
        mat1_sm_out = mat1_cvx.value
        mat2_sm_out = mat2_cvx.value
        Q_sm_out = Q_final_sm
        print(' ')

In [ ]:
choices_maxmin = np.argmax(Q_maxmin_out,1)
choices_am = np.argmax(Q_am_out,1)
baskets_maxmin = {}
baskets_am = {}
for l in range(B1*B2):
    baskets_maxmin[l] = list(np.where(choices_maxmin==l)[0])
    baskets_am[l] = list(np.where(choices_am==l)[0])
cost = np.zeros((B1*B2, B1*B2))
for l in range(B1*B2):
    indices_maxmin = baskets_maxmin[l]
    for l_comparison in range(B1*B2):
        indices_am = baskets_am[l_comparison]
        cost[l, l_comparison] = -len(np.intersect1d(indices_maxmin,indices_am))

In [ ]:
# Python program for the above approach
from scipy.optimize import linear_sum_assignment
row_ind, col_ind = linear_sum_assignment(cost)
match_maxmin_am = -cost[row_ind, col_ind].sum()/N_train
print('relationships preserved @best_found_solutions: %4.4e'%(match_maxmin_am))